In [ ]:
import numpy as np
import zarr
import dask.array as da
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image, write_multiscales_metadata
from ome_zarr.reader import Reader
import mFISHwarp.zarr

In [ ]:
### set parameters
zarr_path = '/mnt/ampa02_data01/tmurakami/kidney/registration/empty.zarr'
physical_scale = (1.0,2.0,1.3,1.3)
downscale_factor = (1,2,2,2)
pyramid_level = 5
axes_info = ['c','z','y','x']
chunk_size = (1,256,256,256)

# Create the zarr group
store = zarr.DirectoryStore(zarr_path, dimension_separator='/')
root = zarr.group(store=store)

In [ ]:
# Create your 4D array data (replace this with your actual data)
data = np.random.rand(4, 512, 512, 512)  # Example data, shape: (C, Z, Y, X)
data = da.from_array(data,chunks=chunk_size)

pyramid = mFISHwarp.zarr.pyramid_generator_from_dask(data, downscale_factor=downscale_factor, pyramid_level=5, chunk=chunk_size)

In [ ]:
# without using ome-zarr library, manually make ome-zarr using pure zarr library and dask
for resolution in range(pyramid_level):
    arr = pyramid[resolution]
    p = root.create_dataset(str(resolution),shape=arr.shape,chunks=chunk_size,dtype=arr.dtype)
    arr.to_zarr(p,dimension_separator='/')

In [ ]:
# write metadata for the zarr using ome-zarr library
datasets = mFISHwarp.zarr.datasets_metadata_generator(physical_scale, downscale_factor, pyramid_level)
write_multiscales_metadata(root, datasets=datasets, axes=axes_info)